In [1]:
import warnings
warnings.filterwarnings('ignore')

데이터 획득

In [2]:
import pandas as pd # 데이터 처리를 위해 판다스 라이브러리를 import 한다.
df = pd.read_csv('./data/basketball_stat.csv') # basketball_stat.csv 파일의 데이터를 데이터프레임으로 불러온다.
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,Player,Pos,3P,2P,TRB,AST,STL,BLK
0,Alex Abrines,SG,1.4,0.6,1.3,0.6,0.5,0.1
1,Steven Adams,C,0.0,4.7,7.7,1.1,1.1,1.0
2,Alexis Ajinca,C,0.0,2.3,4.5,0.3,0.5,0.6
3,Chris Andersen,C,0.0,0.8,2.6,0.4,0.4,0.6
4,Will Barton,SG,1.5,3.5,4.3,3.4,0.8,0.5


In [3]:
df.Pos.value_counts()

C     50
SG    50
Name: Pos, dtype: int64

데이터 시각화

In [ ]:
# 그래프를 출력하기 위해 matplotlib 라이브러리와 seaborn 라이브러리를 import 한다.
import matplotlib.pyplot as plt
import seaborn as sns
# jupyter notebook에 그래프를 출력할 수 있게한다.
%matplotlib inline

In [ ]:
# seaborn 라이브러리의 lmplot() 메소드로 스틸과 2점슛, 어시스트와 2점슛의 포지션(Pos) 별 분포도를 출력한다.
sns.lmplot( data=df, x='STL', y='2P', fit_reg=False, # 데이터, x축, y축, 회귀선 표시 여부
           scatter_kws={'s': 50},                    # 그래프에 표시될 표식의 크기
           markers=['o', 'x'],                       # hue에 따른 표식의 종류
           hue='Pos')                                # 그래프에 표시될 표식의 색상
plt.title('STL and 2P in 2d plane')
sns.lmplot( data=df, x='AST', y='2P', fit_reg=False, scatter_kws={'s': 50}, markers=['o', 'x'], hue='Pos')
plt.title('AST and 2P in 2d plane')
plt.show()

In [ ]:
# seaborn 라이브러리의 lmplot() 메소드로 블로킹과 3점슛, 리바운드와 3점슛의 포지션(Pos) 별 분포도를 출력한다.
sns.lmplot( data=df, x='BLK', y='3P', fit_reg=False, scatter_kws={'s': 50}, markers=['o', 'x'], hue='Pos')
plt.title('BLK and 3P in 2d plane')
sns.lmplot( data=df, x='TRB', y='3P', fit_reg=False, scatter_kws={'s': 50}, markers=['o', 'x'], hue='Pos')
plt.title('TRB and 3P in 2d plane')
plt.show()

데이터 다듬기

In [ ]:
# 판다스의 drop() 메소드로 불필요한 컬럼 전체를 삭제할 수 있다.
# axis 속성은 생략시 0이 기본값이며 행을 삭제하고 1을 쓰면 열을 삭제한다.
# inplace 속성은 False가 기본값이며 메소드가 실행된 결과를 데이터에 반영하지 않고 True를 쓰면 결과를 데이터에 반영한다.
# df = df.drop(['2P', 'AST', 'STL'], axis=1)
df.drop(['2P', 'AST', 'STL'], axis=1, inplace=True)
df.head()

데이터 나누기

In [ ]:
# 학습 데이터와 테스트 데이터로 데이터를 분리하기 위해 sklearn의 train_test_split() 함수를 import 한다.
from sklearn.model_selection import train_test_split
# 데이터의 80%를 학습용 데이터로 데이터의 20%를 테스트 데이터로 무작위로 분리한다.
train, test = train_test_split(df, test_size=0.2)
print(train.shape[0])
print(test.shape[0])

최적의 kNN 파라미터 찾기

In [ ]:
# sklearn의 cross_val_score() 함수의 k-fold 교차 검증을 사용해서 kNN 알고리즘의 조절 가능한 단 하나의 변수 k를 찾는다.
# k-fold 교차 검증 기존 데이터를 k개로 나눠서 k번 정확도를 검증하는 방법이다.
# kNN을 사용하기 위한 라이브러리를 추가한다.
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# 최적의 k를 찾기 위해 교차 검증을 수행할 k의 범위를 3부터 학습 데이터 개수의 절반까지 지정한다.
k_list = []
for i in range(3, train.shape[0] // 2, 2):
    k_list.append(i)
print(k_list)

In [ ]:
# 각각의 k별 10-fold 교차 검증 결과의 평균을 기억할 빈 리스트를 선언한다.
cross_validation_scores = []
# 학습 데이터에서 학습에 사용할 데이터를 저장한다.
x_train = train[['3P', 'TRB', 'BLK']]
# 학습 데이터에서 학습 결과로 사용할 데이터를 저장한다.
y_train = train[['Pos']]

학습 데이터를 10조각으로 나눠 1조각을 검증 데이터로 사용하고 나머지 9조각을 학습 데이터로 사용해서 1번째 조각부터 10번째 조각까지 1번씩 검증하고 10번의 검증 결과를 10으로 나눈 평균을 검증 결과의 점수로 한다.

cross_val_score(model, X, Y, scoring=None, cv=None)  
model : 회귀 분석 모형  
X : 독립 변수 데이터, 학습 데이터  
Y : 종속 변수 데이터, 결과  
scoring : 성능 검증에 사용할 매개 변수에 원하는 평가 지표를 지정한다. => 정확도를 의미하는 'accuracy'를 지정한다.  
cv : 교차 검증 생성기 객체 또는 숫자, None이면 KFold(3), 숫자를 지정하면 KFold(숫자)

In [ ]:
# 10-fold 교차 검증을 각 k를 대상으로 수행해 검증 결과를 저장한다.
for k in k_list:
    # KNeighborsClassifier() 함수의 n_neighbors 속성에 kNN 모델에서 사용할 이웃의 개수를 지정해 kNN 모델을 만든다.
    knn = KNeighborsClassifier(n_neighbors=k)
    # 학습에 따른 결과는 1차원 형태로 지정해야 하므로 values.ravel()를 사용해서 2차원을 1차원으로 변경해야 한다.
    scores = cross_val_score(knn, x_train, y_train.values.ravel(), scoring='accuracy', cv=10)
    cross_validation_scores.append(scores.mean())
print(cross_validation_scores)

각 k별로 출력된 정확도가 쉽게 이해되지 않기 때문에 시각화를 해서 최적의 k를 확인한다.

In [ ]:
plt.plot(k_list, cross_validation_scores)
plt.xlabel('the number of k')
plt.ylabel('accuracy')
plt.show()

In [ ]:
# 예측율이 가장 높은 k를 선정한다.
optimal_k = k_list[cross_validation_scores.index(max(cross_validation_scores))]
# print('최적의 k: {}'.format(optimal_k))
print('최적의 k: ' + str(optimal_k))

모델 테스트

In [ ]:
# 분별력이 있다고 판단된 3점슛(3P), 블로킹(BLK), 리바운드(TRB) 속성으로 모들을 학습한 후 데스트를 진행한다.
knn = KNeighborsClassifier(n_neighbors=optimal_k)
# 학습에 사용할 속성과 예측값을 지정한다.
x_train = train[['3P', 'TRB', 'BLK']]
y_train = train[['Pos']]
knn.fit(x_train, y_train.values.ravel())

# 테스트에 사용할 속성과 포지션에 대한 정답을 지정한다.
x_test = test[['3P', 'TRB', 'BLK']]
y_test = test[['Pos']]

from sklearn.metrics import accuracy_score
# 테스트를 수행한다.
predict = knn.predict(x_test)
# 모델 예측 정확도(accuracy)를 출력한다.
print('정확도: {}'.format(accuracy_score(y_test.values.ravel(), predict)))

In [ ]:
comparison = pd.DataFrame({'예측값': predict, '실제값': y_test.values.ravel()})
comparison

In [ ]:
x  = {'3P': [0, 3.9], 'BLK': [7.7, 0.3], 'TRB': [1, 0.1]}
x_test = pd.DataFrame(x)
print(x_test)
y = {'Pos': ['C', 'SG']}
y_test = pd.DataFrame(y)
print(y_test)

predict = knn.predict(x_test)
print(predict)
print('정확도: {}'.format(accuracy_score(y_test.values.ravel(), predict)))
comparison = pd.DataFrame({'예측값': predict, '실제값': y_test.values.ravel()})
comparison